In [1]:
## Homework: Evaluation and Monitoring

import pandas as pd
url = f'{'https://github.com/varunsood189/llm-zoomcap/blob/main/lesson4/data/results-jamba_1_5_mini.csv'}?raw=1'
df = pd.read_csv(url)
df = df.iloc[:300]


In [3]:
## Q1. Getting the embeddings model
model_name = 'multi-qa-mpnet-base-dot-v1'
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.12.1/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [48]:
answer_llm = df.iloc[0].answer_llm
embedding_model.encode(answer_llm)[0]

np.float32(-0.45160076)


What's the first value of the resulting vector?
* -0.45
## Q2. Computing the dot product


In [11]:
records = df.to_dict(orient='records')

In [44]:
records[0]

{'answer_llm': 'Yes. You can sign up for the course on the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials.',
 'answer_orig': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

In [45]:
from tqdm import tqdm 
evaluations =[]
for data in tqdm(records):
    # print(data["answer_llm"])
    emb_llm=embedding_model.encode(data["answer_llm"])
    emb_orig=embedding_model.encode(data["answer_orig"])
    evaluations+=[ np.dot(emb_llm,emb_orig)]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:17<00:00,  3.86it/s]


In [47]:
np.percentile(evaluations ,75)

np.float32(30.403513)

What's the 75% percentile of the score?
* 31.67

## Q3. Computing the cosine

In [50]:
def norm_v(v):
    norm = np.sqrt((v * v).sum())
    return v / norm
    
evaluations_norm =[]
for data in tqdm(records):
    # print(data["answer_llm"])
    emb_llm=embedding_model.encode(data["answer_llm"])
    emb_orig=embedding_model.encode(data["answer_orig"])
    evaluations_norm+=[ np.dot(norm_v(emb_llm),norm_v(emb_orig))]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:17<00:00,  3.85it/s]


In [51]:
np.percentile(evaluations_norm,75)

np.float32(0.80615664)

What's the 75% cosine in the scores?
np.float32(0.80615664)

## Q4. Rouge

In [52]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [56]:
from rouge import Rouge
rouge_scorer = Rouge()
r = records[10]

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
scores['rouge-1']['f']
# * `rouge-1` - the overlap of unigrams,
# * `rouge-2` - bigrams,
# * `rouge-l` - the longest common subsequence


0.8923076873088758

What's the F score for `rouge-1`?
0.89

## Q5. Average rouge score


In [60]:
avg_f = []
for key in scores.keys():
    avg_f+=[scores[key]['f']]
print(sum(avg_f)/len(avg_f))


0.8704273454284358


Let's compute the average F-score between `rouge-1`, `rouge-2` and `rouge-l` for the same record from Q4
* 0.87

## Q6. Average rouge score for all the data points

In [88]:
evaluations = []
for r in tqdm(records):
    rouge =rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
    evaluations+=[
        {'rouge-1':rouge['rouge-1']['f'],
        'rouge-2':rouge['rouge-2']['f'],
        'rouge-l':rouge['rouge-l']['f'],
        'avergae-rouge':(rouge['rouge-1']['f']+rouge['rouge-2']['f']+rouge['rouge-l']['f'])/2,
        }
    ]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 512.97it/s]


In [89]:
df = pd.DataFrame(evaluations)

In [93]:
df['rouge-2'].mean()

np.float64(0.19068844277809807)

What's the average F-score in `rouge_2` across all the records?
- 0.19